# PDAP-2018: Session 13 (29 June 2018): Advanced Pandas

## Links

### Python Data Science Handbook

- [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html)
- [Combining Datasets: Merge and Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)
- [Aggregation and Grouping](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)
- [Working with Time Series](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html)


## Transcript

Now following is the original transcript as used in class on 29 June 2018.

In [18]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [19]:
planets.head()


,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [27]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [30]:
def my_double_it(df):
    return 2 * df.mean()

planets.groupby('method').apply(my_double_it)['orbital_period']

method
Astrometry                         1262.360000
Eclipse Timing Variations          9503.288889
Imaging                          236495.475000
Microlensing                       6307.142857
Orbital Brightness Modulation         1.418613
Pulsar Timing                     14686.042403
Pulsation Timing Variations        2340.000000
Radial Velocity                    1646.709360
Transit                              42.204145
Transit Timing Variations           159.567000
Name: orbital_period, dtype: float64

In [32]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': np.random.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,0
1,B,1,7
2,C,2,0
3,A,3,7
4,B,4,0
5,C,5,9


In [37]:
df.groupby('key').aggregate(np.min)

,data1,data2
key,,
A,0,0
B,1,0
C,2,0


In [36]:
df.groupby('key').aggregate([np.min, np.median, 'max'])

data1            data2           
     amin median max  amin median max
key                                  
A       0    1.5   3     0    3.5   7
B       1    2.5   4     0    3.5   7
C       2    3.5   5     0    4.5   9

In [38]:
df.groupby('key').aggregate({'data1': np.min, 'data2': np.max})

,data1,data2
key,,
A,0,7
B,1,7
C,2,9


In [40]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,0
1,B,0.142857,7
2,C,0.222222,0
3,A,0.428571,7
4,B,0.571429,0
5,C,0.555556,9


In [42]:
def filter_func(x):
    return x['data2'].std() > 5

df.groupby('key').std()

,data1,data2
key,,
A,2.12132,4.949747
B,2.12132,4.949747
C,2.12132,6.363961


In [43]:
df.groupby('key').filter(filter_func)

,key,data1,data2
2,C,2,0
5,C,5,9


In [45]:
df

,key,data1,data2
0,A,0,0
1,B,1,7
2,C,2,0
3,A,3,7
4,B,4,0
5,C,5,9


In [46]:
def center_it(x):
    return x - x.mean()

df_transformed = df.groupby('key').transform(center_it)
df_transformed['key'] = df['key']
df_transformed

,data1,data2,key
0,-1.5,-3.5,A
1,-1.5,3.5,B
2,-1.5,-4.5,C
3,1.5,3.5,A
4,1.5,-3.5,B
5,1.5,4.5,C


## Hierarchical indexing / MultiIndex

In [50]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=pd.MultiIndex.from_tuples(index))
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [51]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [49]:
pd.MultiIndex.from_tuples(index)

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [52]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [55]:
pop.unstack()

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [56]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [57]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [59]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [68]:
pop.index.names = ['state', 'year']
pop.name = 'population'
pop
# to make a dataframe out of this:
# pd.DataFrame({'population': pop})

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
Name: population, dtype: int64

In [63]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      38.0  37.7  36.0  36.7  32.0  35.3
     2      25.0  36.3  46.0  38.7  23.0  37.4
2014 1      23.0  36.2  45.0  36.3  31.0  38.4
     2      31.0  37.3  34.0  36.5  35.0  36.2

In [69]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      36.0  36.7
     2      46.0  38.7
2014 1      45.0  36.3
     2      34.0  36.5

In [71]:
pop['California', 2010]

37253956

In [72]:
pop['California']

year
2000    33871648
2010    37253956
Name: population, dtype: int64

In [73]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
Name: population, dtype: int64

In [74]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Name: population, dtype: int64

In [77]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
Name: population, dtype: int64

In [82]:
pop[pop > 22000000].unstack()

year,2000,2010
state,,
California,33871648.0,37253956.0
Texas,NaN,25145561.0


In [85]:
pop.unstack('state')

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [90]:
pop_flat = pop.reset_index()
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [91]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [95]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      38.0  37.7  36.0  36.7  32.0  35.3
     2      25.0  36.3  46.0  38.7  23.0  37.4
2014 1      23.0  36.2  45.0  36.3  31.0  38.4
     2      31.0  37.3  34.0  36.5  35.0  36.2

In [94]:
health_data.mean(level='visit')

subject   Bob        Guido         Sue       
type       HR   Temp    HR  Temp    HR   Temp
visit                                        
1        30.5  36.95  40.5  36.5  31.5  36.85
2        28.0  36.80  40.0  37.6  29.0  36.80

In [97]:
health_data.mean(axis='columns', level='type')

type               HR       Temp
year visit                      
2013 1      35.333333  36.566667
     2      31.333333  37.466667
2014 1      33.000000  36.966667
     2      33.333333  36.666667